In [1]:
import pandas as pd
import selenium
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_amazon_url(x):
    x=x.replace(" ","+")
    template="https://www.amazon.in/s?k={}&crid=2YO34STXS8CY4&sprefix={}%2Caps%2C374&ref=nb_sb_noss_1"
    url=template.format(x,x)
    return url

In [3]:
def product_info(product):
    try:
        productDesc=product.find("span","a-size-base-plus a-color-base a-text-normal").text
    except Exception as e:
        productDesc="Not Mentioned"
        
    try:
        productPrice=product.find("span","a-price-whole").text
        productPrice=productPrice.replace("₹","")
        productPrice=productPrice.replace(",","")
    except Exception as e:
        productPrice="Not Mentioned"

    try:
        productName=product.find("span","a-size-base-plus a-color-base").text
    except Exception as e:
        productName="Not Mentioned"
        
    try:
        productImg=product.find("img","s-image")
        productImg=productImg["src"]
    except Exception as e:
        productImg="Not Mentioned"
        
    try:
        productBuyLink=product.find("a","a-link-normal s-no-outline")
        productBuyLink="https://amazon.in/" + productBuyLink["href"]
    except Exception as e:
        productBuyLink="Not Mentioned"
    
#     try:
#         product_description_rating=product.find("span","a-icon-alt").text
#         product_description_rating=product_description_rating[0:3]
#         print(product_description_rating)
#     except Exception as e:
#         product_description_rating="Not Mentioned"
    info=[productName,productPrice,productDesc,productImg,productBuyLink]
    return info

In [4]:
def main(product_name):
    link=get_amazon_url(product_name)
    
    driver=webdriver.Firefox()
    driver.get(link)
    
    soup=BeautifulSoup(driver.page_source,"html.parser")
    products=soup.find_all("div","sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20")
    records=[]
    
    for every_product in products:
        product_details=product_info(every_product)
        records.append(product_details)
   
    col=["productName","productPrice(INR)","productDesc","productImg","productBuyLink"]
    dataset=pd.DataFrame(records,columns=col)
    dataset["productPrice(INR)"]=dataset["productPrice(INR)"].astype("int")
    dataset=dataset.sort_values(by="productPrice(INR)", ascending=False)
    dataset.to_csv("E:\Education\language\WebScrapping Project\Amazon_product_details.csv")
    
#     dataset=dataset.sort_values(by="product_sales_price(INR)",ascending=True)
#     for i in records[0]:
#         print(i)

In [5]:
main("Shoes Black")